# Volume 3: MongoDB.
    Kameron Lightheart
    MATH 405
    1/7/2020

In [57]:
import re
import json
from pymongo import MongoClient
import warnings; warnings.simplefilter('ignore')

## Problem 1

The file `trump.json` contains posts from http://www.twitter.com (tweets) over the course of an hour that have the key word "trump".
Each line in the file is a single JSON message that can be loaded with `json.loads()`.

Create a MongoDB database and initialize a collection in the database.
Use the collection's `delete_many()` method with an empy set as input to clear existing contents of the collection, then fill the collection one line at a time with the data from `trump.json`.
Check that your collection has 95,643 entries with its `count()` method.

In [17]:
client = MongoClient()
db = client.db1
col = db.collection2
col.delete_many({})

In [18]:
with open('trump.json', 'r') as infile:
    for line in infile.readlines():
        json_text = json.loads(line)
        col.insert_one(json_text)

In [61]:
print("Count is:", col.count())

Count is: 95643


## Problem 2

Query the Twitter collection for the following information.
- How many tweets include the word Russia? Use `re.IGNORECASE`.
- How many tweets came from one of the main continental US time zones? These are listed as `"Central Time (US & Canada)"`, `"Pacific Time (US & Canada)"`, `"Eastern Time (US & Canada)"`, and `"Mountain Time (US & Canada)"`.
- How often does each language occur? Construct a dictionary with each language and it’s frequency count.

In [58]:
results = col.find({"text":  {'$regex': re.compile('Russia', re.IGNORECASE)}})
print("There are ", results.count(), "tweets containing the word Russia (Ignoring Case)")

There are  5841 tweets containing the word Russia (Ignoring Case)


In [59]:
results = col.find({"user.time_zone": {"$in": [
    'Central Time (US & Canada)', 
    'Pacific Time (US & Canada)',
    'Eastern Time (US & Canada)',
    'Mountain Time (US & Canada)']}})
print("There are", results.count(), "tweets from the united states time zones")

There are 33939 tweets from the united states time zones


In [60]:
dict = {}
langs = col.distinct('lang')
for lang in langs:
    results = col.find({"lang": lang})
    dict[lang] = results.count()
print(dict)

{'en': 84919, 'fr': 1235, 'es': 3027, 'tr': 588, 'nl': 212, 'pt': 632, 'und': 1653, 'it': 423, 'de': 635, 'da': 35, 'zh': 1, 'ru': 29, 'pl': 108, 'no': 21, 'eu': 6, 'tl': 48, 'in': 121, 'cs': 13, 'ht': 21, 'ro': 21, 'sl': 3, 'sv': 55, 'fi': 20, 'ar': 30, 'et': 23, 'lt': 8, 'hu': 6, 'el': 10, 'ja': 17, 'lv': 2, 'vi': 7, 'fa': 5, 'ko': 2, 'ur': 1, 'cy': 2, 'th': 3, 'is': 1, 'iw': 1, 'uk': 3}


## Problem 3

Query the Twitter collection from Problem 1 for the following information.
- What are the usernames of the 5 most popular (defined as having the most followers) tweeters? Don’t include repeats.
- Of the tweets containing at least 5 hashtags, sort the tweets by how early the 5th hashtag appears in the text. What is the earliest spot (character count) it appears?
- What are the coordinates of the tweet that came from the northernmost location? Use the latitude and longitude point in `"coordinates"`.

In [92]:
results = col.find({"user.followers_count": {"$exists": True}}).sort("user.followers_count", -1)[:10]
names = []
for tweet in results:
    name = tweet["user"]["name"]
    if name not in names:
        print(name)
        names.append(name)
    if len(names) == 5:
        break

CNN
Reuters Top News
Fox News
Wall Street Journal
TIME


In [115]:
results = col.find({"entities.hashtags.4": {"$exists": True}}).sort("entities.hashtags.4.indices.0", 1)
print("Earliest hashtag in tweet is at index:", results[0]["entities"]["hashtags"][4]["indices"][0])

Earliest hashtag in tweet is at index: 23


In [130]:
results = col.find({"coordinates": {"$ne": None}}).sort("coordinates.coordinates.0", -1)
print("Northernmost tweet coordinates:", results[0]["coordinates"]["coordinates"])

Northernmost tweet coordinates: [28.99237342, 41.06803023]


## Problem 4

Clean the Twitter collection in the following ways.

- Get rid of the `"retweeted_status"` field in each tweet.
- Update every tweet from someone with at least 1000 followers to include a popular field whose value is True. Report the number of popular tweets.
- (OPTIONAL) The geographical coordinates used before in coordinates.coordinates are turned off for most tweets. But many more have a bounding box around the coordinates in the place field. Update every tweet without coordinates that contains a bounding box so that the coordinates contains the average value of the points that form the bounding box. Make the structure of coordinates the same as the others, so it contains coordinates with a longitude, latitude array and a type, the value of which should be ’Point’.

In [142]:
col.update_many({}, {"$unset": {"retweeted_status": 1}})
result = col.find({"retweeted_status": {"$exists": True}})
print("Number of tweets with retweeted_status field:", result.count())

Number of tweets with retweeted_status field: 0


In [143]:
col.update_many({"user.followers_count": {"$gte": 1000}}, {"$set": {"popular": True}})
result = col.find({"user.followers_count": {"$gte": 1000}})
print("Does new popular field exist for user with over 1000 followers?", result[0]["popular"])

Does new popular field exist for user with over 1000 followers? True
